In [85]:
import json
import numpy as np
import pandas as pd
import os

from IPython.display import display, Math
from scipy import optimize, stats
from math import exp, cos, sqrt, pi, sin, sqrt, e

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

## Constants to be used across the notebook

In [10]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")    
with open(simulation_values_path, "r") as f:
    simulation_values = json.load(f)
    
simulation_values

{'DT_evap': 8.0,
 'T_base_heat_degree': 15,
 'Tdhw': 55.0,
 'Tmax_i': 60.0,
 'Tmin_i': 45.0,
 'Ts1': 23.0,
 'Ts2': 29.0,
 'cap_source1': 20,
 'cap_source2': 20,
 'depth_aquifer': 30.0,
 'n_buildings_MFH': 1600.0,
 'n_buildings_SFH': 0.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 

In [12]:
load_or_write_new = int(input("Press 1 if you want to reload previous simulation values"))

if load_or_write_new == 1:    
    Tmin_i = simulation_values["Tmin_i"]
    Tmax_i = simulation_values["Tmax_i"]

    Tdhw = simulation_values["Tdhw"]

    Ts1 = simulation_values["Ts1"]
    Ts2 = simulation_values["Ts2"]

    DT_evap = simulation_values["DT_evap"]
    
    n_buildings_SFH = simulation_values["n_buildings_SFH"]
    n_buildings_MFH = simulation_values["n_buildings_MFH"]
    
    cap_source1= simulation_values['cap_source1']
    cap_source2= simulation_values['cap_source2']

    depth_aquifer = simulation_values["depth_aquifer"]
    
    T_base_heat_degree = simulation_values["T_base_heat_degree"]
    
else:
    # Min and Max temperatures to be received by the user.
    # Values [40,...,60]
    Tmin_i = float(input("Enter min setpoint temperature: "))
    Tmax_i = float(input("Enter max setpoint temperature: "))

    # Domestic hot water temperature.
    # Values [45,...,55]
    Tdhw = float(input("Enter domestic hot water temperature: "))

    # Our model supposes that the system has two different sources of heat.
    # Values [20,...,30]
    Ts1 = float(input("Temperature of source 1: "))
    Ts2 = float(input("Temperature of source 2: "))

    # difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
    # Values [1,...,10]
    DT_evap = float(input("Temperature difference among the supply and return pipes: "))
    
    # For the data visualization course, we will select 0 SFH & 1600 MFH buildings, which will cover an area of 1 km2
    n_buildings_SFH =float(input("Enter the number of SFH buildings: #"))
    n_buildings_MFH = float(input("Enter the number of MFH buildings:#"))
    
    cap_source1 = int(input("Enter the thermal capacity of source 1 in MW:"))
    cap_source2 = int(input("Enter the thermal capacity of source 2 in MW:"))

    # The depth of the aquifer in meters. Values are around 30.
    depth_aquifer = float(input("Aquifer temperature depth: "))
    
    T_base_heat_degree = int(input("Heat Degree Days base temperature: "))
    
print(f"* {Tmin_i=}\n"
      f"* {Tmax_i=}\n"
      f"* {Tdhw=}\n"
      f"* {Ts1=}\n"
      f"* {Ts2=}\n"
      f"* {DT_evap=}\n"
      f"* {n_buildings_SFH=}\n"
      f"* {n_buildings_MFH=}\n"
      f"* {cap_source1=}\n"
      f"* {cap_source2=}\n"
      f"* {depth_aquifer=}\n"
      f"* {T_base_heat_degree=}\n"
     )

Press 1 if you want to reload previous simulation values 1


* Tmin_i=45.0
* Tmax_i=60.0
* Tdhw=55.0
* Ts1=23.0
* Ts2=29.0
* DT_evap=8.0
* n_buildings_SFH=0.0
* n_buildings_MFH=1600.0
* cap_source1=20
* cap_source2=20
* depth_aquifer=30.0
* T_base_heat_degree=15



## Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [13]:
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s1_schedule = np.array(simulation_values["s1_schedule"])
else:    
    s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

* s1_schedule_path='./data/private/s1_source_fake_schedule.xlsx'
* s1_schedule=array([[0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0]])
* s1_schedule[6,1]=0
* s1_schedule[7,1]=1



In [14]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s2_schedule = np.array(simulation_values["s2_schedule"])
else: 
    s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

* s2_schedule_path='./data/private/s2_source_fake_schedule.xlsx'
* s2_schedule=array([[1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0]])
* s2_schedule[15,1]=1
* s2_schedule[16,1]=0



## Take note of the values for the current simulation

In [15]:
simulation_values["Tmin_i"] = Tmin_i
simulation_values["Tmax_i"] = Tmax_i
simulation_values["Tdhw"] = Tdhw
simulation_values["Ts1"] = Ts1
simulation_values["Ts2"] = Ts2
simulation_values["DT_evap"] = DT_evap
simulation_values["s1_schedule"] = s1_schedule.tolist()
simulation_values["s2_schedule"] = s2_schedule.tolist()
simulation_values["n_buildings_SFH"] = n_buildings_SFH
simulation_values["n_buildings_MFH"] = n_buildings_MFH
simulation_values['cap_source1']  = cap_source1
simulation_values['cap_source2'] =  cap_source2
simulation_values["depth_aquifer"] = depth_aquifer
simulation_values["T_base_heat_degree"] = T_base_heat_degree

simulation_values

{'DT_evap': 8.0,
 'T_base_heat_degree': 15,
 'Tdhw': 55.0,
 'Tmax_i': 60.0,
 'Tmin_i': 45.0,
 'Ts1': 23.0,
 'Ts2': 29.0,
 'cap_source1': 20,
 'cap_source2': 20,
 'depth_aquifer': 30.0,
 'n_buildings_MFH': 1600.0,
 'n_buildings_SFH': 0.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 

## Read the dataset

In [91]:
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()


AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(noaa2010Dataset.load_data())
AVAILABLE_DATASETS.update(insPireDataset.load_data())

AVAILABLE_DATASETS.keys()

dict_keys(['miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

In [96]:
dataset_to_work = "rome_italy"
dataset_to_work

'rome_italy'

In [97]:
dataset = AVAILABLE_DATASETS[dataset_to_work]
dataset

,hourofyear,air_temp,city_key,climate,DHW_cons,SH_cons,SFH_bldg_tot,MFH_bldg_tot,%SH_y,%DHW_y,DHW_hourly_consumption_ratio,season,date,month,dayofyear,dayofweek,hour
timestamp,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,1,9.100000e+06,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.003885,winter,2017-01-01,1,1,6,0
2017-01-01 01:00:00,2,9.200000e+06,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.003885,winter,2017-01-01,1,1,6,1
2017-01-01 02:00:00,3,9.166667e+06,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.003885,winter,2017-01-01,1,1,6,2
2017-01-01 03:00:00,4,9.183333e+06,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.003885,winter,2017-01-01,1,1,6,3
2017-01-01 04:00:00,5,8.916667e+06,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.003885,winter,2017-01-01,1,1,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,8757,1.550000e+06,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.049895,winter,2017-12-31,12,365,6,20
2017-12-31 21:00:00,8758,7.666667e+05,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.006780,winter,2017-12-31,12,365,6,21
2017-12-31 22:00:00,8759,2.166667e+05,rome_italy,Mediterranean,21.700001,75.900002,97.599998,97.599998,0.777664,0.222336,0.013133,winter,2017-12-31,12,365,6,22


## Definition of the dataset-related constants

- `T_daily_a`: Average air temperature on a daily basis. 
- `DT_y`: Difference between the maximum daily temperature and the mean daily temperature.
- `d_shift_min`: Day of the year with the min read temperature.
- `d_shift_max`: Day of the year with the max read temperature.

In [63]:
# Create a daily dataset
T_daily_avg = dataset["air_temp"].resample('D').mean().to_frame()
T_daily_avg["dayofyear"] = T_daily_avg.index.dayofyear
T_daily_avg["month"] = T_daily_avg.index.month
T_daily_avg["hourofyear"] = (T_daily_avg.index.dayofyear - 1) * 24 + (T_daily_avg.index.hour + 1)

T_ave_h = T_daily_avg.air_temp.mean()
T_max = T_daily_avg.air_temp.max()
DT_y = T_max - T_ave_h

d_shift_min = T_daily_avg.air_temp.idxmin(axis=0).dayofyear #Day of the min Temperature
d_shift_max = T_daily_avg.air_temp.idxmax(axis=0).dayofyear #Day of the max Temperature

t = T_daily_avg.dayofyear

num_hours = dataset.shape[0] # Number of hours in a year = 8737 hrs
h = dataset.hourofyear # A range equivalent to the hours of the year from 1 to 8761

print(f"* {dataset=}\n"
      f"* {T_daily_avg=}\n"
      f"* {T_ave_h=}\n"
      f"* {DT_y=}\n"
      f"* {d_shift_min=}\n"
      f"* {T_daily_avg.air_temp.min()=}\n"
      f"* {d_shift_max=}\n"
      f"* {T_daily_avg.air_temp.max()=}\n"
      f"* {t=}\n"
      f"* {num_hours=}\n"
      f"* {h=}")

* dataset=                     hourofyear  air_temp    city_key         climate  \
timestamp                                                               
2017-01-01 00:00:00           1      7.00  rome_italy  Mediterranean    
2017-01-01 01:00:00           2      9.50  rome_italy  Mediterranean    
2017-01-01 02:00:00           3     11.70  rome_italy  Mediterranean    
2017-01-01 03:00:00           4     11.15  rome_italy  Mediterranean    
2017-01-01 04:00:00           5     10.75  rome_italy  Mediterranean    
...                         ...       ...         ...             ...   
2017-12-31 20:00:00        8757     10.15  rome_italy  Mediterranean    
2017-12-31 21:00:00        8758      9.45  rome_italy  Mediterranean    
2017-12-31 22:00:00        8759      8.75  rome_italy  Mediterranean    
2017-12-31 23:00:00        8760      8.00  rome_italy  Mediterranean    
2018-01-01 00:00:00           1      7.30  rome_italy  Mediterranean    

                      DHW_cons    SH_con

## Functional unit composition

The single-family houses (SFH) and multi-family houses (MFH) models have a fixed geometry for all the climates and periods of construction. It has been defined following the common characteristics for a European buildings. The SFH building model is composed of two storeys with a total of 100 m² of living area . The small-MFH correspond to a reference residential building that has two dwellings per floor, with a total floor area of 500 m2. (Inspire project, D2.1c(2014))

A functional unit of 1 km2 of land composed by a residential area of 1600 buildings of the type small-MFH was selected. This reference area has been already investigated in previous publications for the scenario assessment of low-temperature DH networks, as a typical high-density urban zone.


### Residential building typologies: SFH and MFH

#### Single-Family units

![SFH](images/SFH.png)


#### Multi-Families units

![s-MFH](images/MFH.png)

### District heating system diagram

District heating (DH) is an energy service based on moving heat from available heat sources to immediate customers through a piping network. Nowadays, the fundamental idea of DH is to use LOCAL sources that would otherwise be WASTED in order to satisfy customers'needs (Werner, 2013). 


Traditional DH systems operate typically at temperatures far from the ambient temperature (80°C or higher) giving rise to high thermal losses and the need of costly piping insulation. Examples of traditional DH sources are cogeneration plants, industrial waste heat and incinerators, where heat is a by-product of other industrial processes (Cozzini, 2018).

New concepts in the DH sector propose lowering the operating temperatures further to a level equivalent to the ambient temperature, with an average of 20°C. Advantages of a system of this kind, are the reduction in thermal losses in the distribution system, the direct exploitation of available waste heat sources (refrigeration units in shopping malls, supermarkets, datacenters) and ground sources (aquifer wells), and the possibility of providing not only heating but also cooling through the use of reversible heat pumps on the buildings substations, as presented in the Figure.

![DHC system](images/DHC_system.png)

## 3.0 Data Processing for Space Heating Demand and Domestic Hot Water

The methodology used in this project is a bottom-up approach for the calculation of the total heat demand for space heating (SH) and domestic hot water (DHW) utilization of a district located in 4 different climates across Europe, using data from the Inspire project. The data includes annual energy consumption for each building typology in kWh/m2-y, SFHs and MFHs correspond to reference buildings in each climatic zone built between 1945-1970, non-refurbished. In this particular case, the climatic zones and reference cities considered are:

* Stuttgart, Germany for Continental climate
* London, United Kingdom for Oceanic climate
* Madrid, Spain for Southern-dry climate
* Rome, Italy for Mediterranean climate

This calculation is based on the number of SFH ($ |SFH| $) and MFH ($ |MFH| $) units. It also depends on the total heat needed by SFH ($ TOT_{SFH}$) and MFH ($ TOT_{MFH}$) units. The estimation of the total heated area for buildings within the functional unit ($ HD $) is:

\begin{align*}
HD = |SFH| * 100 * TOT_{SFH} + |MFH| * 500 * TOT_{MFH}
\end{align*}

It's important to stress that this calculation is done on a year basis.

In [ ]:
dataset["Heat_demand"] = (n_buildings_SFH * 100 * dataset["SFH_bldg_tot"]) + (n_buildings_MFH * 500 * dataset["MFH_bldg_tot"])
dataset["SH_demand"] = dataset["Heat_demand"] * dataset["%SH_y"]
dataset["DHW_demand"] = dataset["Heat_demand"] * dataset["%DHW_y"] # Equivalently, we can change dataset["%DHW_y"] with (1 - dataset["%SH_y"])
dataset

## 3.1 Space Heating Profile Distribution

In order to analyse the performance of this particular DH system, it is fundamental not only the spatial distribution of the heat demand, but also the distribution over time. For this purpose, the space heating hourly profile for each location is retrieved with a time dependency according to the heating degree method, known as the "Integration method".

This method accounts the amount (in degrees) and for how long (in hours) thermal heat is required to keep the indoor building temperature at a comfortable level, which will vary depending on different climates. The base temperature selected was set to 15 °C.

More info on calculation method: https://www.degreedays.net/introduction

In [ ]:
def heating_degree(T_amb):
    return max(0, T_base_heat_degree - T_amb)

In [ ]:
dataset["heating_degree_days"] = dataset.air_temp.apply(heating_degree)
dataset["space_heating_dist"] = dataset.heating_degree_days / max(sum(dataset.heating_degree_days), 1)
dataset

## 3.2 Space Heating Consumption (H)

In [ ]:
dataset["SH_consumption"] = dataset["SH_demand"] * dataset["space_heating_dist"]
dataset

## 3.3 Domestic Hot Water Consumption (H)

Domestic hot water consumption is not weather influenced, and its variation is almost constant over the year. The hourly demand for hot water is estimated by the percentage of the total heat demand ($ HD_y $) is hot water $ DHW_y $. This data is also obtained from the INSPIRE database for different climatic zones. 

The total domestic hot water demand is evenly distributed over each day of the year, and then its hourly distribution is obtained by multiplying the daily needs by an hourly random profile. 

\begin{align*}
DHW_{d} = \frac{HD_{y}*DHW_{\%y}}{d}
\end{align*}

The DHW consumption is calculated in an hourly fashion.

In [ ]:
dataset["DHW_consumption"] = dataset["DHW_demand"] * dataset["DHW_hourly_consumption_ratio"] / 365
dataset

## 3.4 Thermal Load Profile (H) 

Thus, the aggregated heat demand corresponds to the sum of the thermal heat requirements for SH and DHW, according to the equation:

\begin{align*}
HD_{TOT, h} = HD_{SH, h} + HD_{DHW, h}
\end{align*}

In [ ]:
dataset["Total_consumption"] = dataset["DHW_consumption"] + dataset["SH_consumption"]
dataset

### 3.4.1 Adjusting the total consumption by air temperature

In [ ]:
dataset["Total_consumption_fit"] = np.nan

records_with_heat_degree_days = dataset[dataset.air_temp <= T_base_heat_degree]
if records_with_heat_degree_days.shape[0] == 0:
    print(f"Key {dataset_to_work} has no degree days.")
else:
    slope, intercept, r_value, p_value, std_err = stats.linregress(records_with_heat_degree_days.air_temp, records_with_heat_degree_days.Total_consumption)
    dataset["Total_consumption_fit"] = slope * records_with_heat_degree_days.air_temp + intercept

dataset

## 3.5 Climate Curve (H)

Temperature levels required for SH and DHW. The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [ ]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [ ]:
dataset['space_heating_temp'] = dataset.air_temp.apply(climatic_curve)
dataset["hot_water_temp"] = Tdhw
dataset

## 3.6 User Temperature Level (H)

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [ ]:
sh_consumption_ratio = dataset["SH_consumption"] / dataset["Total_consumption"]
dhw_consumption_ratio = dataset["DHW_consumption"] / dataset["Total_consumption"]

In [ ]:
dataset["user_temp"] = (sh_consumption_ratio * dataset.space_heating_temp 
                        + dhw_consumption_ratio * dataset.hot_water_temp)
dataset

## 3.7 Fitting Curve (H)

For more info visit the [scipy.optimize.curve_fit docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit)

In [ ]:
def func(x, disp, amp, phi):
    t0 = len(h) # hourly resolution if x is in hours
    omega = 2 * pi / t0
    return disp + amp * np.cos(x * omega - phi)

x_data = pd.Series(range(0,len(dataset)))
y_data = dataset.air_temp
params, params_covariance = optimize.curve_fit(func, x_data, y_data, p0=[T_ave_h, DT_y, d_shift_max * 24])
print(f"* {params=}\n"
      f"* {params_covariance=}")

In [ ]:
T_ave_fit = params[0]
DT_y_fit = params[1]
phi = params[2] #Corresponds to day 202

#'displacement, amplitude, and phase of the signal
print('Fitted parameters:')
display(Math(f"dist={T_ave_fit:.2f}, amp={DT_y_fit:.2f}, \\phi={phi:.2f}"))

print('Original parameters:')
display(Math(f"dist={T_ave_h:.2f}, amp={DT_y:.2f}, \\phi={d_shift_max*24:.2f}"))

### 3.7.1 Calculate the fitting curve

Using the fitted parameters we can calculate the fitted air temperature

In [ ]:
def fitting_curve(hourofyear, T_ave_fit, DT_y_fit, T, phi, pi):
    omega = 2 * pi / T
    return T_ave_fit + DT_y_fit * np.cos(hourofyear * omega - phi)

In [ ]:
T = dataset.hourofyear.shape[0]
dataset["air_temp_fit"] = dataset.hourofyear.apply(fitting_curve, args=(T_ave_fit, DT_y_fit, T, phi, pi))

d_shift_min = dataset.air_temp_fit.idxmin(axis=0)
d_shift_max = dataset.air_temp_fit.idxmax(axis=0)

dd_min = dataset[dataset.index == d_shift_min].dayofyear.item() # d_shift_min / 24
dd_max = dataset[dataset.index == d_shift_max].dayofyear.item() # d_shift_max / 24

print(f'* {T=}\n'
      f'* {dataset[["air_temp_fit"]]=}\n'
      f'* d_min (h): {d_shift_min}\n'
      f'* day_number: {dd_min}\n'
      f'* d_max (h): {d_shift_max}\n'
      f'* day_number: {dd_max}')

## 3.8 Ground temperature

The ground temperature is calculated as a function of the ambient temperature fitting curve using the Kusuda equation, ref: 
- [Measurements of Ground Temperature at Various Depths](https://www.researchgate.net/publication/30500372_Measurements_of_Ground_Temperature_at_Various_Depths?enrichId=rgreq-e2031a024b742c0018bb428dca3100f5-XXX&enrichSource=Y292ZXJQYWdlOzMwNTAwMzcyO0FTOjEwMTI0NTIzNDcxMjU4M0AxNDAxMTUwMTUzNjYy&el=1_x_2&_esc=publicationCoverPdf)
- [Earth Temperatures and Thermal Diffusivity at Selected Stations in the United States](https://nvlpubs.nist.gov/nistpubs/Legacy/RPT/nbsreport8972.pdf)

In [ ]:
zz = 1 # Depth [m]
alpha = 0.06048 # Ground thermal diffusivity, Banks [m^2/day]
alpha_sec = 7e-7 # Ground thermal diffusivity, Banks [m^2/s]
t_sec = 365 * 24 * 3600 # Number of seconds in a year.

t_0 = T_daily_avg.shape[0] # Number of days in a year =365 days

Tg_und = T_ave_fit # Undisturbed ground temperature
DT_y = abs(DT_y_fit)

print(f"* {zz=}\n"
      f"* {alpha=}\n"
      f"* {alpha_sec=}\n"
      f"* {t_sec=}\n"
      f"* {t_0=}\n"
      f"* {Tg_und=}\n"
      f"* {DT_y=}\n"
      f"* {dd_min=}\n"
      f"* {dd_max=}")

In [ ]:
def ground_temperature_hour(t: int):
    #t is time in hours, but the calculation is done is seconds
    kusuda = Tg_und - DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec))) *cos(2*pi/t_sec*((t-dd_min*24)*3600-zz/2*sqrt(t_sec/(pi*alpha_sec)))) #Kusuda
    return pd.Series({'T_kusuda': kusuda})

In [ ]:
dataset["ground_temp"] = dataset.hourofyear.apply(ground_temperature_hour)
dataset

## 3.9 Aquifer Well Temperature (Y)

We use the same equation (Kusuda) to calculate the temperature of an aquifer well located `depth_aquifer` meters depth.

In [ ]:
zz = depth_aquifer

print(f"* {zz=}\n"
      f"* {alpha=}\n"
      f"* {alpha_sec=}\n"
      f"* {t_sec=}\n"
      f"* {t_0=}\n"
      f"* {Tg_und=}\n"
      f"* {DT_y=}\n"
      f"* {dd_min=}\n"
      f"* {dd_max=}")

In [ ]:
dataset["aquifer_temp"] = dataset.hourofyear.apply(ground_temperature_hour)
dataset


## 3.10 Network temperature (H)

### 3.10.1 Calculate Sources Temperature based on Working Hours

In [ ]:
dayofweeks = dataset.index.dayofweek
hours = dataset.index.hour

In [ ]:
dataset["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
dataset["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]
dataset

### 3.10.2 Calculate Network Temperature

In [ ]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.min([temp_s1, temp_s2])

In [ ]:
dataset["net_temp"] = dataset.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)
dataset

## 3.11 Coefficient of performance (COP)

Heat pumps located at the users' substations are responsible for lifting the network temperature to a desired level for SH and DHW consumptions. The performance of the overall system was assessed according to the following coefficient of performance (COP) function:

\begin{equation}
COP\ = \eta_{m} COP_{C} +1 - \eta_{m}
\end{equation}

\begin{equation}
COP_{C}= \frac{T_{c}}{T{_c} - T_{e}} 
\end{equation}

\begin{equation}
T_{c}= T_{c,o} +  \Delta T_{HEX} 
\end{equation}

\begin{equation}
T_{e}=  T_{e,o} -  \Delta T_{HEX} 
\end{equation}

Where $COP_{C}$ corresponds to the Carnot COP which is a function of the condenser ($ T_{c}$) and evaporator ($T_{e}$) refrigerant temperatures respectively. These variables are estimated as the external fluid outlet temperatures, adjusted by a temperature drop ($\Delta T_{HEX}$) at the heat pump condenser and evaporator.  

In [ ]:
DT_hx = 2.5
n_HP = 0.53
Te_o = dataset.net_temp - DT_evap - DT_hx
Tc_o = dataset.user_temp

dataset['COP'] = 1 - n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)
dataset

## 3.12 Heat losses

In [ ]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 0.848  #Average heat loss of pre-insulated pipes along a network of 2km [MW/ K] 
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U  # Heat losses supply pipe [MW]
    HL_r = (T_ret - T_gr) * U  # Heat losses return pipe [MW]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [ ]:
dataset[['E_loss_s', 'E_loss_r', 'E_loss_tot']] = dataset.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')
dataset

## 3.13 Electricity consumption

The hourly electricity consumption by climatic zone is estimated as a function of the total thermal demand and the coefficient of performance of the systems substations:

\begin{align*}
E_{el} = \frac{E_{th}}{COP}
\end{align*}

In [ ]:
dataset['E_el'] = dataset['Total_consumption'] / dataset['COP']
dataset

## 3.14 Thermal Energy from Sources

District heating systems have an opportunity to use heat supply from several sources at the same time. In order for this to happen, it is necessary a control system that allows many thermal plants to deliver heat at the same distribution system. In this case, the base load can be supplied by up to two sources of waste heat, with a maximum capacity and temperature levels determined by the user of this tool. A third source consisting in aquifer wells provides heat from a depth of about 30 m when the base load plants are not operating, as well as in peak load conditions. 



In [ ]:
dataset["source1_cap"] = cap_source1 * 1000 * s1_schedule[hours, dayofweeks] # values in kW to match the thermal demand calculations
dataset["source2_cap"] = cap_source2 * 1000 *  s2_schedule[hours, dayofweeks] # values in kW to match the thermal demand calculations
dataset

In [ ]:
def calculate_heatsupply(source1_cap, source2_cap, source1_temp, source2_temp,thermal_energy_from_sources):
    if source1_cap == 0.0 and source2_cap == 0.0:
        heat_source1 = 0.0
        heat_source2= 0.0
        heat_aquifer = thermal_energy_from_sources
        return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])    
    elif source1_cap == 0.0:                    #Si la primera fuente no funciona, la otra cubre la demanda hasta su max capacidad
        if thermal_energy_from_sources <=  source2_cap:
            heat_source1 = 0.0
            heat_source2 = thermal_energy_from_sources
            heat_aquifer = 0.0
        else:
            heat_source1 = 0.0
            heat_source2 = source2_cap
            heat_aquifer = thermal_energy_from_sources - source2_cap
        return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])    
    elif source2_cap == 0.0:                                 #Si la segunda fuente no funciona, la otra cubre la demanda hasta su max capacidad
        if thermal_energy_from_sources <=  source1_cap:
            heat_source1 = thermal_energy_from_sources
            heat_source2 = 0.0
            heat_aquifer = 0.0
        else:
            heat_source1 = source1_cap
            heat_source2 = 0.0
            heat_aquifer = thermal_energy_from_sources - source1_cap
        return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])    
    else:                            #Si ambas plantas funcionan, opera la de mayor temperatura hasta su max capacidad, si no es suficiente, se activa la segunda planta
        if source1_temp < source2_temp:
            
            if thermal_energy_from_sources <=  source1_cap:
                heat_source1 = thermal_energy_from_sources
                heat_source2 = 0.0
                heat_aquifer = 0.0
            else:
                heat_source1 = source1_cap
                heat_source2 = thermal_energy_from_sources - source1_cap
                heat_aquifer = 0.0
            
            return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])           
        else: 
            if thermal_energy_from_sources <=  source2_cap:
                heat_source1 = 0.0
                heat_source2 = thermal_energy_from_sources
                heat_aquifer = 0.0
            else:
                heat_source1 = thermal_energy_from_sources - source2_cap
                heat_source2 = source2_cap
                heat_aquifer = 0.0
            return pd.Series([heat_source1, heat_source2, heat_aquifer] , index=['heat_source1','heat_source2','heat_aquifer'])

In [ ]:
dataset[["heat_source1", "heat_source2", "heat_aquifer"]] = dataset.apply(lambda fila: calculate_heatsupply(fila.source1_cap,fila.source2_cap, fila.source1_temp, fila.source2_temp, fila.Total_consumption - fila.E_el), axis=1 , result_type='expand')
dataset

## Save CSV version of the dataset and simulation values

In [ ]:
AVAILABLE_DATASET_PATHS = {NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path,
                           InsPireDataset.LONDON_UK: insPireDataset.processed_london_uk_dataset_path,
                           InsPireDataset.MADRID_SPA: insPireDataset.processed_madrid_spa_dataset_path,
                           InsPireDataset.ROME_IT: insPireDataset.processed_rome_it_dataset_path,
                           InsPireDataset.STUTTGART_GER: insPireDataset.processed_stuttgart_ger_dataset_path,}

dataset.to_csv(path_or_buf=f"{AVAILABLE_DATASET_PATHS[dataset_to_work]}.csv")
dataset.to_parquet(path=f"{AVAILABLE_DATASET_PATHS[dataset_to_work]}.parquet", engine="pyarrow")

In [ ]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")

with open(simulation_values_path, "w") as f:
    json.dump(simulation_values, f, sort_keys=True, indent=2)